In [ ]:
import polars as pl
# Cargamos el dataset que ya existe
df = pl.read_csv("dataset/amazon_sales_dataset.csv")

# Comprobamos que los datos se cargaron correctamente
df.head()

order_id,order_date,product_id,product_category,price,discount_percent,quantity_sold,customer_region,payment_method,rating,review_count,discounted_price,total_revenue
i64,str,i64,str,f64,i64,i64,str,str,f64,i64,f64,f64
1,"""2022-04-13""",2637,"""Books""",128.75,10,4,"""North America""","""UPI""",3.5,443,115.88,463.52
2,"""2023-03-12""",2300,"""Fashion""",302.6,20,5,"""Asia""","""Credit Card""",3.7,475,242.08,1210.4
3,"""2022-09-28""",3670,"""Sports""",495.8,20,2,"""Europe""","""UPI""",4.4,183,396.64,793.28
4,"""2022-04-17""",2522,"""Books""",371.95,15,4,"""Middle East""","""UPI""",5.0,212,316.16,1264.64
5,"""2022-03-13""",1717,"""Beauty""",201.68,0,4,"""Middle East""","""UPI""",4.6,308,201.68,806.72


In [ ]:
# 1. Extraemos los valores como arreglos de NumPy
y = df['total_revenue'].to_numpy()
X = df['discounted_price'].to_numpy()

# 2. Estandarizamos X (Restamos la media y dividimos por la desviación estándar)
X_mean = X.mean()
X_std = X.std()
X_scaled = (X - X_mean) / X_std

print(f"Variables listas: {len(y)} registros cargados.")

# Explicación para el grupo:
# La estandarización convierte los precios (que pueden ser cientos o miles) en valores que suelen estar entre -3 y 3. Esto evita que los cálculos matemáticos del modelo "exploten" o tarden demasiado en encontrar la solución.

Variables listas: 50000 registros cargados.
